---
Minimal Network (Euler 107), or why matroid are useful. 
---

[This problem](https://projecteuler.net/problem=107) asks us to find a minimum spanning tree (MST) in a weighted network, then calculate the savings by pruning the graph to this MST. 

We'll implement Kruskal's algorithm, which seems to be a classic in the canon of algorithms that CS majors learn.  However it is simply a particular case of the **greedy algorithm** for maximizing a weight function over a [matroid](https://en.wikipedia.org/wiki/Matroid). In this case, the matroid $M$ is the collection of sets of edges not containing a cycle, and the bases of this matroid are the spanning trees.

It is a theorem of Rado that the greedy algorithm produces an optimal set over a family of sets $M$ if an only if $M$ is a matroid. In fact, the greedy algorithm completely classifies classical matroids. Let's verify this for the very specific case where we are using the greedy algorithm to construct a minimum spanning tree.  

Our ground set in this case is the set of edges $E = \{ (v,w) \in V\times V \}$ of a graph $G$ on $V$, and we call a set of edges *independent* if it does not contain a cycle. Maximal indpendent sets, or bases, are spanning trees $\mathcal{T}$.     In this context, the **basis exchange property** says that given two trees, $T$ and $S$, if $g\in T \setminus S$ then there should be an $f\in S\setminus T$ such that $T+f-g$ is also a tree. Let's verify that $\mathcal{T}$ is indeed a basis of a matroid. 

* *If $e\in S\setminus T $ then removing it creates connected components partitioning $Vert(G)=A\sqcup B$.   Since $T$ is a spanning tree, there must an edge $f\in T$ connecting $A$ and $B$, however this edge cannot be $S$ since $e$ is the only bridge between these two components in $S$.  Therefore $S-e+f$ is once again a spanning tree. In fact $T-f+e$ is also still a tree.*

So the set of spanning trees actually satisfies a **strong exchange property** and does indeed form a matroid.  This will give us a bit more machinery to prove that the greedy algorithm will produce an optimum. We'll use the fact that any independent set $I$ of a matroid can be extended to a basis $I\subset B$.   
 
The greedy algorithm (Kruskal's) is absurdly unsophisticated.  Since we are minimizing, we start by adding the smallest element to our set $T$.  At every iteration, we add the smallest element $e$ such that $T+e$ is stil independent (doesn't contain cycles).  And since this is a matroid, we know that at every iteration, our set of edges  $T_i$ is contained in a tree $B_i$.  

Let $w: E \rightarrow \bf{R}^+$ be the weight function on the edges.  Suppose $i$ is the first time where we make a set $T_i$ that is not contained in an optimal base $B_i$. Let $e$ be the element we add at this iteration.  Since $T_{i-1}\subset B_{min}$ where $B_{min}$ is a MST, we can use the strong exchange property to replace an $f\in B_{min}\setminus T_{i-1}$ with $e$. However all such $f$ will have equal or larger weight than $e$ since $e$ minimizes $w(T_{i-1}+e)$ over all $e$ not forming a cycle.  But the minimality of $B_{min}$ implies that $w(f) \leq w(e)$ proving that $w(B_i) = w(B_{min})$ for all $i$.  

Ok, so now it's safe to use the greedy algorithm.   
 
 


In [4]:
# Assuming you have the data file handy...

edges = []
network_data = file("../data/p107_network.txt",'r')
lines = [l.split(",") for l in network_data.readlines()]

n = len(lines[0])

total_sum = 0
for i in range(n):
    for j in range(i):
        if lines[i][j] != '-':
            total_sum += int(lines[i][j])
            edge = (int(lines[i][j]), i, j)
            edges.append(edge)

edges.sort()

trees = [set([x]) for x in range(n)]


min_sum = 0
for (w,n1,n2) in edges:
    tree1 = None
    tree2 = None
    for i in range(len(trees)):
        if n1 in trees[i]:
            tree1 = i
        if n2 in trees[i]:
            tree2 = i
            
    if tree1 == tree2:
        continue
    else:
        min_sum += w
        trees[tree1] = trees[tree1].union(trees[tree2])
        del trees[tree2]
        
print "Minimum weight: " + str(min_sum)
print "Maximum Savings: " + str(total_sum - min_sum)

Minimum weight: 2153
Maximum Savings: 259679
